Imports

In [3]:
import os
import asyncio
import google.generativeai as genai
from sklearn.model_selection import train_test_split

data preprocessing

In [6]:
source_path_for_bbc = r"C:\Users\elias\Documents\NLP and speech processing"



def process_folders(folder_path):
    bbc_data = {}
    first_documents = {}

    # Iterate over all folders in the provided folder path
    for folder_name in os.listdir(folder_path):
        folder_full_path = os.path.join(folder_path, folder_name)

        # Ensure we are processing only folders
        if os.path.isdir(folder_full_path):
            documents = os.listdir(folder_full_path)
            # Split the dataset into training and test sets
            train, test = train_test_split(documents, test_size=0.2, random_state=42)

            print(f"Processing folder: {train[folder_name][0]}")
            first_documents[folder_name] = train[folder_name][0]

            for test_doc in test:
                with open(os.path.join(folder_full_path, test_doc), 'r') as file:
                    text = file.read()
                    bbc_data[folder_name].append(text)
    for i in first_documents:
        print(i, first_documents[i])

    for j in test:
        print(j, test[j])



    return first_documents, bbc_data

#Data Structure: Topic, Article Text

bbc_instructions, bbc_data = process_folders(source_path_for_bbc)


Processing folder: business


TypeError: list indices must be integers or slices, not str

Make Prompt

In [20]:
def make_prompts(bbc_instructions, bbc_data):
    prompts = []
    general_instruction = "You are a perfect topic modeling machine. Given a text and the different topics, you will classify the texts to the correct topic. First you will receive the topics, afterwards an example and finally the text you have to assign one of the before mentioned topics to."
    topics = "The topics are business, entertainment, politics, sport and tech. Please make sure, you know the topics and their meaning."
    transition_to_examples = "Now a example for each of the categories will follow."
    
    transition_to_text_to_classify = "Now the text, you have to classify will follow. Please assess its topic and answer only the topic of it. "

    for entry in bbc_data:

        for text in bbc_data[entry]:
            prompt = general_instruction + "\n" +topics + "\n" + transition_to_examples + "\n"

            for instruction in bbc_instructions:
                prompt += "For the following text: \n" + bbc_instructions[instruction]+ " \nThe correct answer would be:" +  instruction+ "\n"

            prompt += transition_to_text_to_classify + "\n" + text + "\n"
            prompts.append(prompt)

    return prompts            

prompts = make_prompts(bbc_instructions, bbc_data)

print(prompts[0])

You are a perfect topic modeling machine. Given a text and the different topics, you will classify the texts to the correct topic. First you will receive the topics, afterwards an example and finally the text you have to assign one of the before mentioned topics to.
The topics are business, entertainment, politics, sport and tech. Please make sure, you know the topics and their meaning.
Now a example for each of the categories will follow.
For the following text: 
Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it no

In [ ]:
semaphore = asyncio.Semaphore(10)

async def prompt_llm(prompt,  time_interval):
    async with semaphore:
        name = list(prompt.keys())[0]
        promptAI = prompt[name]   #this is the prompt

        generation_config = {
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 8192,
        "response_mime_type": "text/plain",
        }
        model = genai.GenerativeModel(
        model_name="gemini-1.5-pro",
        generation_config=generation_config,
        )
        chat_session = model.start_chat(
        history=[
        ]
        )
        answer = chat_session.send_message(promptAI)
        await asyncio.sleep(time_interval)
        
        return {name : answer}